In [2]:
import cv2
import mediapipe as mp
import face_recognition
import numpy as np
import os
import firebase_admin
from firebase_admin import credentials,messaging
from firebase_admin import firestore
from datetime import datetime, timedelta
import time
import threading

In [4]:
cred = credentials.Certificate("esp32-9e776-firebase-adminsdk-24lzj-b0eb618d5a.json")
app  = firebase_admin.initialize_app(cred)

def sendPush(title, msg, registration_token, dataObject=None):
    # See documentation on defining a message payload.
    message = messaging.MulticastMessage(
        notification=messaging.Notification(
            title=title,
            body=msg
        ),
        data=dataObject,
        tokens=registration_token,
    )

    response = messaging.send_multicast(message)
    print('Successfully sent message:', response)   

In [3]:
def get_Token():
    db = firestore.client()
    doc = db.collection('tokens').get()
    tokens = []
    for i in doc:
        tokens.append(i.to_dict()['fcmToken'])
    return tokens

In [4]:
def camera_system():
    global door_unlocked
    known_faces_dir = 'PROJECT/p1'
    known_face_encodings, known_face_names = load_known_faces(known_faces_dir)

    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils

    # Initialize webcam
    cap = cv2.VideoCapture(0)

    # Dictionary to track the last recognized time
    last_recognized_time = {name: datetime.min for name in known_face_names}

    with mp_face_detection.FaceDetection(
            model_selection=1, min_detection_confidence=0.5) as face_detection:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the frame to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(rgb_frame)

            face_detected = False

            if results.detections:
                for detection in results.detections:
                    # Get bounding box
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                int(bboxC.width * iw), int(bboxC.height * ih)
                    # Draw bounding box
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                    # Extract face ROI
                    face_roi = np.ascontiguousarray(rgb_frame[y:y + h, x:x + w])

                    # Recognize the face using face_recognition
                    face_encodings = face_recognition.face_encodings(face_roi)
                    if face_encodings:
                        face_encoding = face_encodings[0]
                        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                        best_match_index = np.argmin(face_distances)
                        current_time = datetime.now()
                        if matches[best_match_index] and (current_time - last_recognized_time[known_face_names[best_match_index]] > timedelta(minutes=1)):
                            name = known_face_names[best_match_index]
                            last_recognized_time[name] = current_time
                            tokens = get_Token()
                            sendPush("Known Person Detected", f"{name} Detected", tokens)
                            save_recognized_face(face_roi, name)
                            db = firestore.client()
                            doc = db.collection('flags').document('lock').set({'yes':False})
                            print("Face detected. Pausing for one minute.")
                            door_unlocked=1
                            
                        else:
                            name = "Unknown"
                            tokens = get_Token()
                            sendPush("Unknown Person Detected", "Unknown Person Detected", tokens)
                            save_recognized_face(face_roi, name)
                            print("Face detected. Pausing for one minute.")
                            time.sleep(60)
                        
                        # Draw name
                        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                        while(door_unlocked):
                                print("Door Opened")
                                door_unlocked = int(input('Enter State: '))


            cv2.imshow('MediaPipe Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()


In [5]:
thread1 = threading.Thread(target=camera_system)
thread1.start()

c:\Users\nithi\College\4TH SEMESTER\INTRODUCTION TO COMMUNICTION AND IOT\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Successfully sent message: <firebase_admin.messaging.BatchResponse object at 0x000001727FBA7990>
C:/Users/nithi/Documents/IoT/p1_20240528_011237.jpg
Door Opened
Enter State: 

: 

In [8]:
def door_lock_change():
    global door_unlocked
    db = firestore.client()

    doc = db.collection('flags').document('lock').get()

    print(doc.to_dict())

door_lock_change()

{'yes': True}
